# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing dataset!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing data again. This time, however, we did things slightly different:
- We dropped `'ZN'` and `'NOX'` completely 
- We categorized `'RAD'` in 3 bins and `'TAX'` in 4 bins
- We transformed `'RAD'` and `'TAX'` to dummy variables and dropped the first variable.
- We used min-max-scaling on `'B'`, `'CRIM'`, and `'DIS'` (and logtransformed all of them first, except `'B'`)
- We used standardization on `'AGE'`, `'INDUS'`, `'LSTAT'`, and `'PTRATIO'` (and logtransformed all of them first, except for `'AGE'`) 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(['NOX', 'ZN'],axis=1)

# First, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0, 6, 24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# First, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix='TAX', drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix='RAD', drop_first=True)
boston_features = boston_features.drop(['RAD','TAX'], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features['AGE']
b = boston_features['B']
logcrim = np.log(boston_features['CRIM'])
logdis = np.log(boston_features['DIS'])
logindus = np.log(boston_features['INDUS'])
loglstat = np.log(boston_features['LSTAT'])
logptratio = np.log(boston_features['PTRATIO'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['CRIM'] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['AGE'] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features['INDUS'] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features['PTRATIO'] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [23]:
boston_features.columns = ['CRIM','INDUS','CHAS','RM','AGE','DIS','PTRATIO','B','LSTAT','RAD_6_24','TAX_270_360','TAX_360_712']


## Run an linear model in Statsmodels

In [35]:
# Your code here
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [38]:
X = boston_features
y = pd.DataFrame(boston.target, columns = ['price'])

In [42]:
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()

In [43]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Sun, 13 Oct 2019   Prob (F-statistic):          5.08e-153
Time:                        21:31:11   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.6442      3.189      2.711      0.007       2.379      14.910
CRIM           -1.9538      2.115     -0.924      0.356      -6.110       2.202
INDUS          -0.8046      0.362     -2.220      0.027      -1.517      -0.093
CHAS            2.5959      0.796      3.260      0.001       1.032       4.160
RM              2.6466      0.408      6.488      0.000       1.845       3.448
AGE             0.0794      0.352      0.226      0.821      -0.612       0.770
DIS           -10.0962      1.856     -5.439      0.000     -13.743      -6.449
PTRATIO        -1.4867      0.241     -6.160      0.000      -1.961      -1.013
B               3.8412      0.986      3.897      0.000       1.905       5.778
LSTAT          -5.6288      0.354    -15.912      0.000      -6.324      -4.934
RAD_6_24        1.3380      0.672      1.990      0.047       0.017       2.659
TAX_270_360    -1.2598      0.600     -2.100      0.036      -2.438      -0.081
TAX_360_712    -2.1461      0.704     -3.047      0.002      -3.530      -0.762
==============================================================================
Omnibus:                      106.730   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              432.101
Skew:                           0.891   Prob(JB):                     1.48e-94
Kurtosis:                       7.162   Cond. No.                         117.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Run the same model in Scikit-learn

In [44]:
# Your code here - Check that the coefficients and intercept are the same as those from Statsmodels
from sklearn.linear_model import LinearRegression

In [45]:
model2 = LinearRegression()

In [47]:
model2 = model2.fit(X,y)


array([8.64415614])

In [49]:
model2.intercept_
model2.coef_

array([[  0.        ,  -1.95380233,  -0.80457549,   2.59586776,
          2.64657111,   0.07939727, -10.09618465,  -1.48666599,
          3.8412139 ,  -5.62879369,   1.33796317,  -1.25977612,
         -2.14606188]])

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [50]:
model2.predict([.15,6.07,1,6.1,33.2,7.6,17,383,10.87,8,284])

ValueError: Expected 2D array, got 1D array instead:
array=[1.500e-01 6.070e+00 1.000e+00 6.100e+00 3.320e+01 7.600e+00 1.700e+01
 3.830e+02 1.087e+01 8.000e+00 2.840e+02].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing data.